In [1]:
from abc import ABCMeta, abstractmethod
from multiprocessing import Pool
from functools import partial
import numpy as np
from numba import jit

In [2]:
class loss(metaclass=ABCMeta):
    '''
    Absctract base class for loss function.
    
        pred() takes GBDT outputs, i.e., the "score" as its inputs, and returns predictions.
        g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
        h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.

    All inputs and outputs are numpy arrays.
    '''
    @abstractmethod
    def pred(self,score):
        pass

    @abstractmethod
    def g(self,true,score):
        pass

    @abstractmethod
    def h(self,true,score):
        pass

#TODO: loss of least square regression and binary logistic regression
class mse(loss):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
        return score-true

    def h(self,true,score):
        return np.ones_like(score)

class log(loss):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
        return 1.0/(1.0+np.exp(-score))

    def g(self,true,score):
        pred=self.pred(score)
        return pred-true

    def h(self,true,score):
        pred=self.pred(score)
        return pred*(1.0-pred)

In [19]:
# TODO: class of Random Forest
class RF(object):
    '''
    Class of Random Forest
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        rf = 0.5, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.num_trees = num_trees
        self.rf = rf

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        
        if self.loss == 'mse':
            self.loss = mse()
        if self.loss == 'log':
            self.loss = log()
        n = len(target)
            
        self.trees = []
        self.score_start = target.astype('float').mean()
        # score is the accumulated prediction $y^k$ and should be updated once a tree has been added
        score = np.ones(len(train)) * self.score_start
        for i in range(self.num_trees):
            
            # bootstrap sampling (random sampling with replacement)
            index = np.random.choice(n, n)
            
            tree = Tree(n_threads = self.n_threads,
                      max_depth = self.max_depth, min_sample_split = self.min_sample_split,
                      lamda = self.lamda, gamma = self.gamma, rf = self.rf)
            tree.fit(train[index], g = self.loss.g(target[index], score[index]), h = self.loss.h(target[index], score[index]))
            self.trees.append(tree)
            
            print('tree', i)
            
        return self

    def predict(self, test):
        
        score = np.ones(len(test)) * self.score_start
        for i in range(self.num_trees):
            score += (1.0/float(self.num_trees)) * self.trees[i].predict(test)
        return self.loss.pred(score)

In [4]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        learning_rate = 0.1, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        
        if self.loss == 'mse':
            self.loss = mse()
        if self.loss == 'log':
            self.loss = log()
            
        self.trees = []
        self.score_start = target.astype('float').mean()
        # score is the accumulated prediction $y^k$ and should be updated once a tree has been added
        score = np.ones(len(train)) * self.score_start
        for i in range(self.num_trees):
            
            tree = Tree(n_threads = self.n_threads,
                      max_depth = self.max_depth, min_sample_split = self.min_sample_split,
                      lamda = self.lamda, gamma = self.gamma)
            tree.fit(train, g = self.loss.g(target, score), h = self.loss.h(target, score))
            self.trees.append(tree)
            
            score += self.learning_rate * tree.predict(train)
            print('tree', i)
            
        return self

    def predict(self, test):
        
        score = np.ones(len(test)) * self.score_start
        for i in range(self.num_trees):
            score += self.learning_rate * self.trees[i].predict(test)
        return self.loss.pred(score)

In [5]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.
    
    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.
    
    A node on a tree can be either a leaf node or a non-leaf node.

    Parameters:
        is_leaf: If is TreeNode is a leaf, i.e., node $j\in L_k$ or $j\in N_k$ in hw.
        score: The prediction (score) of a tree leaf, the $w^k_j$ in hw.
        split_feature: The split feature of a tree node, the $p_j$ in hw.
        split_threshold: The split threshold of a tree node, the $\tau_j$ in hw.
        left_child: Pointing to a child TreeNode, 
                    where the value of split feature is less than the split threshold.
        right_child: Pointing to a child TreeNode, 
                    where the value of split features is greater than or equal to the split threshold.
    '''
    
    def __init__(self,
        is_leaf = False, score = None,
        split_feature = None, split_threshold = None,
        left_child = None, right_child = None):
        
        self.is_leaf = is_leaf
        self.score = score
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left_child = left_child
        self.right_child = right_child

In [26]:
# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
    '''
    
    def __init__(self, n_threads = None, 
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0, nb = False, rf = 0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.nb = nb
        self.rf = rf

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        self.n, self.m = train.shape
        self.tree = self.construct_tree(train, g, h, self.max_depth)
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        Multiprocessing is supported for prediction.
        '''
        pool = Pool(self.n_threads)
        f = partial(self.predict_single, self.tree)
        result = np.array(pool.map(f, test))
        pool.close()
        pool.join()
        return result

    def predict_single(self, treenode, test):
        '''
        The predict method for a single sample point.
        test must be numpy array (a m-dim vector)
        Return prediction (score) as a number.
        '''
        if treenode.is_leaf:
            return treenode.score
        else:
            if test[treenode.split_feature] < treenode.split_threshold:
                return self.predict_single(treenode.left_child, test)
            else:
                return self.predict_single(treenode.right_child, test)

    def construct_tree(self, train, g, h, max_depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.
        
        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
            
        The above stopping conditions indicate and cover two extra stopping conditions:
            4. Targets take only one value.
            5. Each feature takes only one value.
            
            By careful design, we could avoid checking condition 4 and 5 explicitly.
            In function find_threshold(), the best_gain is set to 0 initially.
            So if there are no further feature to split,
            or all the targets take the same value,
            the return value of best_gain would be zero.
            Thus condition 3 would be satisfied,
            and no further splitting would be done.
            To conclude, we need only to check condition 1,2 and 3.
        '''
        # stopping condition 1 & 2
        if max_depth == 0 or len(train) < self.min_sample_split:
        # we start from max_depth, and reduce it by 1 everytime when we split a node, 
        # so max_depth == 0 means the tree reaches the maximum depth
            return TreeNode(is_leaf = True, score = self.leaf_score(g, h))
        
        feature, threshold, gain = self.find_best_decision_rule(train, g, h)

        # stopping condition 3
        if gain <= self.gamma:
        # the gain here used is different from hw in that it does not includes the last -λ
            return TreeNode(is_leaf = True, score = self.leaf_score(g, h))

        index = train[:, feature] < threshold
        left_child = self.construct_tree(train[index], g[index], h[index], max_depth-1)
        right_child = self.construct_tree(train[~index], g[~index], h[~index], max_depth-1)
        
        return TreeNode(split_feature = feature, split_threshold = threshold, 
                        left_child = left_child, right_child = right_child)
    
    def leaf_score(self, g, h):
        '''
        Given the gradient and hessian of a tree leaf node,
        return the optimal weight $w^k_j$ at this leaf, which is -G/(H+λ).
        '''
        return -np.sum(g)/(np.sum(h)+self.lamda)

    def leaf_loss(self, g, h):
        '''
        Given the gradient and hessian of a tree leaf node,
        return the minimized loss at this leaf, which is -0.5*G^2/(H+λ).
        '''
        return -0.5*np.square(np.sum(g))/(np.sum(h)+self.lamda)

    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j, 
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train
        
        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        
        # find best threshold per feature
        pool = Pool(self.n_threads)
        f = partial(self.find_threshold, g, h)
        if self.rf > 0:
            rand_subset = np.random.choice(self.m, int(self.rf * self.m), replace = False)
            thresholds = np.array(pool.map(f, train.T[rand_subset]))
        else:
            thresholds = np.array(pool.map(f, train.T))
        pool.close()
        pool.join()
        
        # find the best feature with the largest gain
        feature=np.argmax(thresholds[:, 1], axis=0)
        threshold=thresholds[feature, 0]
        gain=thresholds[feature, 1]
        if self.rf > 0:
            feature = rand_subset[feature]
        
        return feature, threshold, gain
    
    def find_threshold(self, g, h, train):
        '''
        Given a particular feature $p_j$,
        return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        
        # precompute 1st term of the gain
        loss = self.leaf_loss(g, h)
        
        # initialization
        threshold = None
        best_gain = 0
        
        # avoid same feature values from different data points (samples)
        unq=np.unique(train)
        
        # compare all the possible thresholds
        for i in range(1, len(unq)):
            
            # try one threshold and compute gain
            this_threshold = (unq[i-1] + unq[i])/2
            
            index = train < this_threshold
            left_loss = self.leaf_loss(g[index], h[index])
            right_loss = self.leaf_loss(g[~index], h[~index])
            # first three terms of the gain (except the last -λ)
            this_gain = loss - left_loss - right_loss
            
            if this_gain > best_gain:
                threshold = this_threshold
                best_gain = this_gain
                
        return [threshold, best_gain]

In [10]:
# TODO: Evaluation (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
    rmse = np.sqrt(np.mean((pred - y)**2))
    return rmse

# precision
def accuracy(pred, y):
    pred = pred > 0.5
    return sum(np.equal(pred, y)) / float(len(y))

In [15]:
# test GBDT
model = GBDT(n_threads=8, loss='mse', 
             max_depth=3, min_sample_split = 10, 
             lamda = 1.0, gamma = 0.1,
             learning_rate=0.1, num_trees=100)
train = np.random.randn(1000, 50)
target = np.random.randn(1000)
model.fit(train, target)

In [16]:
pred = model.predict(train)

In [18]:
print('RMSE =', root_mean_square_error(pred, target))

RMSE = 0.6856408799133467


In [43]:
# TODO: GBDT regression on boston house price dataset

# load data
from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [8]:
# train GBDT regression on boston
model = GBDT(n_threads=4, loss='mse', 
             max_depth=5, min_sample_split = 10, 
             lamda = 2.0, gamma = 0.1,
             learning_rate=0.1, num_trees=30)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19
tree 20
tree 21
tree 22
tree 23
tree 24
tree 25
tree 26
tree 27
tree 28
tree 29


In [11]:
# evaluate GBDT regression on boston
print('GBDT Training RMSE =', root_mean_square_error(model.predict(X_train), y_train))
print('GBDT Test RMSE =', root_mean_square_error(model.predict(X_test), y_test))

GBDT Training RMSE = 1.616135667597893
GBDT Test RMSE = 3.576038913706025


In [ ]:
# train RF regression on boston
model = RF(n_threads=4, loss='mse', 
             max_depth=5, min_sample_split = 10, 
             lamda = 2.0, gamma = 0.1,
             rf = 0.5, num_trees=30)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18


In [45]:
# evaluate RF regression on boston
print('RF Training RMSE =', root_mean_square_error(model.predict(X_train), y_train))
print('RF Test RMSE =', root_mean_square_error(model.predict(X_test), y_test))

RF Training RMSE = 3.5786261133089026
RF Test RMSE = 4.101972771458844


In [42]:
y_pred = model.predict(X_test)
print(y_test[:10], y_pred[:10])

[1 0 0 1 1 1 1 1 0 1] [0.86323301 0.57355438 0.64687948 0.87353876 0.76004527 0.58468808
 0.91684898 0.83928138 0.53350157 0.5666469 ]


In [30]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [31]:
# train GBDT classification on breast_cancer
model = GBDT(n_threads=4, loss='log', 
             max_depth=5, min_sample_split = 10, 
             lamda = 1.0, gamma = 0.1,
             learning_rate=0.1, num_trees=20)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19


In [32]:
# evaluate GBDT classification on brest cancer
print('GBDT Training accuracy =', accuracy(model.predict(X_train), y_train))
print('GBDT Test accuracy =', accuracy(model.predict(X_test), y_test))

GBDT Training accuracy = 0.9974874371859297
GBDT Test accuracy = 0.9649122807017544


In [33]:
# train RF classification on breast_cancer
model = RF(n_threads=4, loss='log', 
             max_depth=5, min_sample_split = 10, 
             lamda = 1.0, gamma = 0.1,
             rf = 0.3, num_trees=20)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19


In [34]:
# evaluate RF classification on brest cancer
print('RF Training accuracy =', accuracy(model.predict(X_train), y_train))
print('RF Test accuracy =', accuracy(model.predict(X_test), y_test))

RF Training accuracy = 0.9874371859296482
RF Test accuracy = 0.9415204678362573


In [35]:
# TODO: GBDT classification on credit-g dataset
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [36]:
# train GBDT regression on credit-g
model = GBDT(n_threads=4, loss='log', 
             max_depth=5, min_sample_split = 10, 
             lamda = 2.0, gamma = 0.2,
             learning_rate=0.3, num_trees=30)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19
tree 20
tree 21
tree 22
tree 23
tree 24
tree 25
tree 26
tree 27
tree 28
tree 29


In [37]:
# evaluate GBDT classification on credit-g
print('GBDT Training accuracy =', accuracy(model.predict(X_train), y_train))
print('GBDT Test accuracy =', accuracy(model.predict(X_test), y_test))

GBDT Training accuracy = 0.9657142857142857
GBDT Test accuracy = 0.7866666666666666


In [38]:
# train RF regression on credit-g
model = RF(n_threads=4, loss='log', 
             max_depth=5, min_sample_split = 10, 
             lamda = 2.0, gamma = 0.2,
             rf = 0.3, num_trees=30)
model.fit(X_train, y_train)

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19
tree 20
tree 21
tree 22
tree 23
tree 24
tree 25
tree 26
tree 27
tree 28
tree 29


In [39]:
# evaluate GBDT classification on credit-g
print('RF Training accuracy =', accuracy(model.predict(X_train), y_train))
print('RF Test accuracy =', accuracy(model.predict(X_test), y_test))

RF Training accuracy = 0.7957142857142857
RF Test accuracy = 0.7633333333333333
